<a href="https://colab.research.google.com/github/razevortex/PyDBin/blob/main/TypeSystems0_95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install bitarray and adaptations

In [21]:
!pip install bitarray

In [22]:
DEBUG_MODE = 0
def cout(*args, on=1):
    if DEBUG_MODE == on:
        print(' > '.join([str(arg) for arg in args]))

In [23]:
# @title Import and Adaptations

#############################################################################################################################################################################
# The Adaptations are
#
# bitarray.bitarray => <class bit>
#   - fromfile -> readf and is a classmethod instead of the read binIO the argument it needs is the file path, it accounts to the new _pad method
#   - tofile -> writef is changed similar to readf its just not a classmethod, it uses the added _pad method
#   + _pad = since the bits written to a file will be padded to bytes _pad adds 1 byte to the start with an equal amount of 1´s in it as there are padded 0´s to the end
#
# bitarray.utils =>
#   int2ba -> i2b now returns bit instead of bitarray
#   ba2int -> b2i now returns bit instead of bitarray
##############################################################################################################################################################################

from bitarray import bitarray as ba
from bitarray.util import int2ba, ba2int
from random import randint as rng


class bit(ba):
    @classmethod
    def readf(cls, file):
        temp = cls()
        with open(file, 'rb') as f:
            temp.fromfile(f)
        return temp[8:-temp[:8].count(1)]

    def cut(self, bits:int):
        temp, self[:] = self[:bits], self[bits:]
        return temp

    def _pad(self):
        i = 0
        pad = [[0 for _ in range(8)], []]
        while (len(self) + i) % 8 != 0:
            pad[0][i] = 1
            pad[1].append(0)
            i += 1
        return bit(pad[0]) + self + bit(pad[1])

    def writef(self, file):
        temp = self if len(self) % 8 == 0 else self._pad()
        with open(file, 'wb') as f:
            temp.tofile(f)

    def rng_array(self, size:int, chance:tuple):
        self.size = size
        return bit([int(rng(0, chance[1]) > chance[0]) for i in range(size)])


def i2b(val:int, len:int, bit_class=bit):
    return bit_class(int2ba(val, len))


def b2i(bits:bit):
    return ba2int(bits)



# TypeEnv

In [24]:
# @title Type Enviroment


class DynamicTypes(object):
    __slots__ = ('bit_int', 'bit_str', 'bit_bool', 'bit_none', 'bit_float', 'bit_list', 'bit_tuple', 'bit_dict')
    def __init__(self):
        [self.__setattr__(slot, globals()[slot]) for slot in self.__slots__]

    @property
    def bits(self):
        return 8

    @property
    def _iterative(self):
        return [self.__getattribute__(slot)() for slot in self.__slots__]

    def write_bits(self, value):
        for i, _t in enumerate(self._iterative):
            if _t == value:
                return i2b(i, self.bits) + _t.write_bits(value)
        return bit()

    def read_bits(self, bits):
        i = b2i(bits.cut(self.bits))
        return self._iterative[i].read_bits(bits)


class StaticTypes(object):
    __slots__ = ('sint', 'uint', 'sfloat', 'sstr', 'tuple_T', 'list_T', 'dict_T') #, 'keywords', 'T_list_n', 'T_tuple_n', 'T_dict_n')
    def __init__(self, *args):
        [self.__setattr__(slot, globals()[slot]) for slot in self.__slots__]
        for arg in args:
            self.__getattribute__(arg[0])(arg[1])

    # This Part is for storeing/loading the defined types to and from files
    @property
    def header(self):
        temp = DynamicTypes().write_bits(self.definition)
        return DynamicTypes().write_bits(len(temp)) + temp

    @classmethod
    def from_file(cls, file):
        length = DynamicTypes().read_bits(file)
        return cls(*DynamicType.read_bits(file.cut(length)))

    @property
    def definition(self):
        return [item.definition for item in self.type_list]

    def build_definition(self, k, v):
        return self.__getattribute__(k)(v)

    # This Part is for the actual read/write of the types values
    @property
    def type_list(self):
        temp = []
        slots = [self.__getattribute__(slot) for slot in self.__slots__]
        for slot in slots:
            for define in slot.DEFINED:
                if not (str(slot.__name__).startswith('tuple') or str(slot.__name__).startswith('list') or str(slot.__name__).startswith('dict')):
                    temp.append(slot(define))
                else:
                    temp.append(slot([self.build_definition(k, v) for k, v in define]))
        return temp

    @property
    def _iterative(self):
        return [t for t in self.type_list]

    @property
    def bits(self):
        return len(self._iterative)

    def get_type(self, key):
        for item in self._iterative:
            if item.name == key:
                return item

class Objects(object):
    __slots__ = 'objects'
    def __init__(self, *args):
        self.objects = [self.add_obj(globals()[arg]) for arg in args]

    @property
    def definition(self):
        return [obj.name for obj in self.objects]

    @property
    def header(self):
        temp = DynamicTypes().write_bits(self.definition)
        return DynamicTypes().write_bits(len(temp)) + temp

    def __len__(self):
        return len(self.objects)

    @classmethod
    def from_file(cls, file):
        length = DynamicTypes().read_bits(file)
        return cls(*DynamicType.read_bits(file.cut(length)))

    def add_obj(self, obj):
        self.objects.append(obj())

    #
    #def add_obj(obj):
    #    class obj_(object):
    #        def __init__(self, _obj):
    #            self.name = _obj.__name__
    #            self.obj = _obj

    #        def write_bits(self, obj):
    #            return obj.write_bits()

    #        def read_bits(self, bits):
    #            return self.obj.read_bits(bits)

    #    return obj_(obj)

    @property
    def _iterative(self):
        return [obj for obj in self.objects]

    @property
    def bits(self):
        return len(self.objects)

class Env(object):
    def __init__(self, name, file=None):
        self.name = name
        self.dynamic = DynamicTypes()
        if not file is None:
            self.static = StaticTypes.from_file(file)
            self.objects = Objects.from_file(file)
        else:
            self.static = StaticTypes()
            self.objects = Objects()
        self.buffer = []

    def add_buffer(self, type_, item):
        self.buffer += [(type_, item)] # type_ = the type class name attribute that should be used to handle it; item = the actual object that should be stored

    @property
    def _iterative(self):
        return self.dynamic._iterative + self.static._iterative + self.objects._iterative

    @property
    def bits(self):
        i = 0
        while 2 ** i <= self.dynamic.bits + self.static.bits + len(self.objects):
            i += 1
        return i

    def declare_static(self, type_:str, definition):
        self.static.__getattribute__(type_)(definition)

    def declare_object(self, type_:type):
        self.objects.add_obj(type_)

    def __repr__(self):
        msg = f'Env {self.name}:\n'
        for i, t in enumerate(self._iterative):
            msg += f'{i}. {t.name}\n'
        return msg

    @property
    def listing(self):
        return [t for t in self._iterative]

    def get_type(self, key):
        for t in self.listing:
            if t.name == key:
                return t

    def write(self):
        temp = bit()
        for _t, item in self.buffer:
            for i, T in enumerate(self.listing):
                if T.name == _t:
                    temp += i2b(i, self.bits)
                    temp += T.write_bits(item)
        return temp

    def read(self, bits):
        arr = []
        while len(bits) > 1:
            temp = self.listing[b2i(bits.cut(self.bits))]
            arr.append(temp.read_bits(bits))
        return arr

    @property
    def body(self):
        temp = bit()
        for _t, item in self.buffer:
            for T in self._iterative:
                if T.name == _t:
                    temp += T.write_bits(item)
        return self.dynamic.write_bits(len(temp)) + temp



In [25]:
# @title Length encoding
#############################################################################################################################
# a specification of types is the way their length is determind, there are 2 different approaches
#
#   length_encoded_type - define a amount of bits at the start that can be decoded to get the length
#
#   endbit_encoded_type - a reapiting block with a set length with a specific block that marks the endpoint
#
#   static_encoded_type - a type with a set static length with no overhead except of the own declaration
#
# the types inherit the behavior from these classes
#############################################################################################################################
class length_encoded_type(object):
    DEFINED = None
    __name__ = ''
    __slots__ = 'name', 'bits', 'encoding', 'decoding'
    def __init__(self, bits):
        self.bits = bits
        self.encoding = self._encoding
        self.decoding = self._decoding
        self.name = self.__name__

    @property
    def definition(self):
        return bit()

    def _encoding(self, *args):
        # defined in Child
        return None

    def _decoding(self, *args):
        # defined in Child
        return None

    def _read(self, *args):
        return None

    def _write(self, *args):
        return None

    def read_bits(self, bits:bit):
        length = self.encoding(bits.cut(self.bits))
        return self._read(bits.cut(length))

    def write_bits(self, value):
        length = self.decoding(value)
        if not length is None:
            return self._write(value, length)

    def conversion(self, in_):
        if isinstance(in_, bit):
            return self.read_bits(in_)
        return self.write_bits(in_)

class endbit_encoded_type(object):
    DEFINED = None
    __name__ = ''
    __slots__ = 'name', 'bits', 'stop_sequence'
    def __init__(self, bits, stop_sequence):
        self.bits, self.stop_sequence = bits, stop_sequence
        self.name = self.__name__

    @property
    def definition(self):
        return bit()

    def read_bits(self, bits:bit):
        i = 0
        temp, _t = bit(), bits.cut(self.bits)
        while _t != self.stop_sequence and len(_t) == self.bits:
            temp += _t
            _t = bits.cut(self.bits)
        return temp

    def write_bits(self, value):
        return value + self.stop_sequence

    def conversion(self, in_):
        if isinstance(in_, bit):
            return self.read_bits(in_)
        return self.write_bits(in_)


# some predefined set of types with static lengths
class static_encoded_type(object):
    __name__ = ''
    DEFINED = None
    def_type = None
    __slots__ = 'name', 'bits'
    def __init__(self, bits:int|bit):
        if isinstance(bits, bit):
            bits = b2i(bits)
        self.bits = bits
        self.name = self.__name__

    @property
    def definition(self):
        return bit()

    def _read(self, bits:bit):
        return None

    def _write(self, value):
        return None

    def read_bits(self, bits:bit):
        return self._read(bits.cut(self.bits))

    def write_bits(self, value):
        return self._write(value)

    def conversion(self, in_):
        if isinstance(in_, bit):
            return self.read_bits(in_)
        return self.write_bits(in_)

def add_define(_type, definition):
    if definition not in _type.DEFINED:
        _type.DEFINED.append(definition)

In [26]:
# @title Dynamic Types (Standalone)

## These Types are used for writing/reading the header data

## bit_int is decoding int with 2 ** 4-bit length encoding it can handle size up to 32768 bit => 2 ** 32768 => dec 9865 digits well exceeding the default python limit of 4700
## bit_int is most crucial since it contains the decoded length of itself also its signed all that makes it adjust to the value but its going to be much slower then the static ints

## but it is going to be used to initialize these defining there length

class bit_int(length_encoded_type):
    __name__ = 'int'
    def __init__(self, *args):
        super().__init__(4)

    def __eq__(self, other):
        if type(other) == int:
            return True
        else:
            return False

    def _encoding(self, bits:bit):
        return [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768][b2i(bits)] + 1 #2 ** b2i(bits) + 1 # returns the length in bits of the integer

    def _read(self, bits:bit):
        sign = bits.cut(1)
        return b2i(bits) if b2i(sign) == 0 else -b2i(bits)

    def _decoding(self, val:int):
        for i, v in enumerate([1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768]):
            if 2 ** v > abs(val):
                return i
        print(f'ERROR {val} is to large')

    def _write(self, value:int, length:int):
        temp = bit([0]) if value >= 0 else bit([1])
        return i2b(length, self.bits) + temp + i2b(abs(value), 2**length)


# char is not a type on its own its more a char set used by str types
class bit_schar(static_encoded_type):
    def __init__(self):
        super().__init__(8)
        self.chars = [chr(i) for i in range(256)]
        self.name = 'native_char'

    def _read(self, bits:bit):
        return chr(b2i(bits.cut(self.bits)))

    def _write(self, val):
        return i2b(ord(val), self.bits)

class bit_str(endbit_encoded_type):
    __name__ = 'str'
    def __init__(self, *args):
        self.chars = bit_schar()
        super().__init__(self.chars.bits, i2b(0, self.chars.bits))

    def __eq__(self, other):
        if type(other) == str:
            return True
        else:
            return False

    def read_bits(self, bits:bit):
        bits = super().read_bits(bits)
        string = ''
        while len(bits) >= self.bits:
            string += self.chars.read_bits(bits)
        return string

    def write_bits(self, val):
        temp = bit()
        for char in str(val):
            temp += self.chars.write_bits(char)
        return super().write_bits(temp)


class bit_fchar(static_encoded_type):
    def __init__(self):
        super().__init__(4)
        self.chars = [char for char in '1234567890.-: '] # The ' ' and ':' are included since there where these unused values anyway and like this it can be used for datetimes later on
        self.name = 'fchar'

    def _read(self, bits):
        return self.chars[b2i(bits.cut(self.bits))]

    def _write(self, val):
        return i2b(self.chars.index(val), self.bits)

class bit_float(endbit_encoded_type):
    __name__ = 'float'
    def __init__(self, *args):
        self.chars = bit_fchar()
        super().__init__(self.chars.bits, i2b(15, 4))

    def __eq__(self, other):
        if type(other) == float:
            return True
        else:
            return False

    def read_bits(self, bits):
        bits = super().read_bits(bits)
        return float(''.join([self.chars.read_bits(bits) for _ in range(len(bits) // self.bits)]))

    def write_bits(self, val):
        temp = bit()
        for char in str(val):
            temp += self.chars.write_bits(char)
        return super().write_bits(temp)

class bit_bool(static_encoded_type):
    __name__ = 'bool'
    def __init__(self, *args):
        super().__init__(1)

    def __eq__(self, other):
        if type(other) == bool:
            return True
        else:
            return False

    def _read(self, bits:bit):
        return (False, True)[b2i(bits)]

    def write_bits(self, val):
        return i2b(int(val), self.bits)

class bit_none(static_encoded_type):
    __name__ = 'NoneType'
    def __init__(self, *args):
        super().__init__(0)

    def __eq__(self, other):
        if other is None:
            return True
        else:
            return False

    def read_bits(self, bits:bit):
        return None

    def write_bits(self, val):
        return bit()

class bit_list(endbit_encoded_type):
    __name__ = 'list'
    types = DynamicTypes

    def __init__(self, *args):
        self.types = bit_list.types()
        super().__init__(1, i2b(0, 1))

    def __eq__(self, other):
        if type(other) == list:
            return True
        else:
            return False

    def read_bits(self, bits:bit):
        temp = []
        while bits.cut(self.bits) != self.stop_sequence:
            temp.append(self.types.read_bits(bits))
        return temp

    def write_bits(self, value):
        temp = bit()
        for item in value:
            print(item)
            temp += i2b(1, self.bits) + self.types.write_bits(item)
        return temp + self.stop_sequence
        return temp + self.stop_sequence

class bit_tuple(endbit_encoded_type):
    __name__ = 'tuple'
    types = DynamicTypes
    def __init__(self):
        self.types = bit_tuple.types()
        super().__init__(1, i2b(0, 1))

    def __eq__(self, other):
        if type(other) == tuple:
            return True
        else:
            return False

    def read_bits(self, bits:bit):
        temp = []
        while bits.cut(self.bits) != self.stop_sequence:
            temp.append(self.types.read_bits(bits))
        return tuple(temp)

    def write_bits(self, value):
        temp = bit()
        for item in value:
            temp += i2b(1, self.bits) + self.types.write_bits(item)
        return temp + self.stop_sequence


class bit_dict(endbit_encoded_type):
    __name__ = 'dict'
    types = DynamicTypes
    def __init__(self):
        self.types = bit_dict.types()
        super().__init__(1, i2b(0, 1))

    def __eq__(self, other):
        if type(other) == dict:
            return True
        else:
            return False

    def read_bits(self, bits:bit):
        temp = {}
        while bits.cut(self.bits) != self.stop_sequence:
            key = self.types.read_bits(bits)
            temp[key] = self.types.read_bits(bits)
        return temp

    def write_bits(self, value):
        temp = bit()
        for key, val in value.items():
            temp += i2b(1, self.bits) + self.types.write_bits(key) + self.types.write_bits(val)
        return temp + self.stop_sequence



In [27]:
# @title Type Class Test

def TEST(type_, val, definition=8, long_=False):
    val = val if type(val) == tuple else (val, )
    print(f'{type_.__name__} tests:')
    for v in val:
        bit_ = type_(definition).definition + type_(definition).write_bits(v)
        temp = bit([b for b in bit_])
        val_ = type_(temp).read_bits(temp)
        if long_:
            print(f'\tValue: {v} => write_bits => {bit_} => read_bits => {val_} remaining bits = {temp}')
        else:
            print(f'\t{v}: Vals={"PASSED" if v == val_ else "FAILED"} 0Bits={"PASSED" if len(temp) == 0 else "FAILED"}')
    print('\n')

# Type Base Classes

**Some Insights to my Thoughts so far:**

After setting the basic enviroment, the first base type´s are introduced ints floats and boolean/none
that float shows up here actualy is a dedactical mishap since i use a stringlike way to handle them and while a char (a single symbol of a string) is handled like its decoded integer representation in C the actual string is more an open array of them.

I want to note that the way of my length/endbit/static encoding is not 1:1 copied from another language ( at least i didn´t know of ) but its cruecial to have some method to now where to split the endless string of '1' and '0' that binary is.

Many languages have a set of static types int, long int, int_8, also float and double are useualy bound to a specific bit-length, and there are good reasons to do so. The amount of overhead ( bits needed to identifiy type, its length, meta stuff that not realy is related to the stored value ) is just an short identifier what makes it way faster then dynamic types.
Later with implementing container like arrays/lists this leads to the massive speed losses of python for loops to c.

What i tried to do is making static versions dynamicly define static types to prevent beeing bound to a fixed set.



In [28]:
# @title Static Int

## bit_int is decoding int with 2 ** 4-bit length encoding it can handle size up to 32768 bit => 2 ** 32768 => dec 9865 digits well exceeding the default python limit of 4700
## bit_int is most crucial since it contains the decoded length of itself also its signed all that makes it adjust to the value but its going to be much slower then the static ints

## but it is going to be used to initialize these defining there length


class uint(static_encoded_type):
    __name__ = 'uint_'
    DEFINED = []
    def __init__(self, bits:int|bit):
        if isinstance(bits, bit):
            bits = b2i(bits)
        super().__init__(bits)
        add_define(uint, self.bits)
        self.name = self.__name__ + str(self.bits)

    @property
    def definition(self):
        # the definition property will be writen in the header to initialize specific types
        return [type(self).__name__, self.bits] #static_bit_int.def_type().write_bits(self.bits)

    def _read(self, bits):
        return b2i(bits)

    def _write(self, val):
        if val < 2 ** self.bits:
            return i2b(val, self.bits)
        else:
            print(f'ERROR {val} > {2 ** self.bits}')


class sint(static_encoded_type):
    __name__ = 'sint_'
    DEFINED = []
    def __init__(self, bits:int|bit):
        if isinstance(bits, bit):
            bits = b2i(bits)
        super().__init__(bits)
        add_define(sint, self.bits)
        self.name = self.__name__ + str(self.bits)

    @property
    def definition(self):
        # the definition property will be writen in the header to initialize specific types
        return [type(self).__name__, self.bits]

    def _read(self, bits):
        sign = bits.cut(1)
        return b2i(bits) if b2i(sign) == 0 else -b2i(bits)

    def _write(self, val):
        if abs(val) < 2 ** (self.bits - 1):
            temp = bit([0]) if val >= 0 else bit([1])
            return temp + i2b(val, self.bits - 1)
        else:
            print(f'ERROR {val} > {2 ** (self.bits - 1)}')



In [29]:
# @title Static Float

class sfloat(static_encoded_type):
    __name__ = 'sfloat_'
    DEFINED = []
    def __init__(self, bits):
        if isinstance(bits, bit):
            bits = b2i(bits)
        super().__init__(4 * bits)
        add_define(ufloat, bits)
        self.chars = bit_fchar()

    def _read(self, bits):
        return float(''.join([self.chars.read_bits(bits) for _ in range(len(bits) // 4)]))

    def _write(self, val):
        temp = bit()
        val = str(val).lstrip('0') + '0' * (self.bits // 4)
        for i in range(self.bits // 4):
            temp += self.chars.write_bits(val[i])
        return temp

In [30]:
# @title Static Str/Keywords


class sstr_list(list):
    def __init__(self):
        super().__init__()

    def append(self, other):
        if type(other) != tuple:
            return
        try:
            a = self.pop(0)
        except:
            a = ()
        temp = []
        [temp.append(string) for string in a + other if string not in temp]
        super().append(tuple(temp))

class sstr(static_encoded_type):
    __name__ = 'sstr'
    DEFINED = sstr_list()
    def __init__(self, bits:tuple|bit):
        self.name = 'sstr'
        if isinstance(bits, bit):
            self.words = DynamicTypes().read_bits(bits)
        sstr.DEFINED.append(bits)
        self.words = sstr.DEFINED[0]

    @property
    def definition(self):
        return (type(self).__name__, self.words)

    @property
    def bits(self):
        i = 0
        while 2 ** i <= len(self.words):
            i += 1
        return i

    def _write(self, value):
        print(self.words)
        if type(value) == str:
            return i2b(self.words.index(value), self.bits)

    def _read(self, bits):
        return self.words[b2i(bits)]


In [31]:
# @title Static List


class tuple_T(static_encoded_type):
    DEFINED = []
    __name__ = 'tuple_T_'
    def __init__(self, *_items):
        self.items = [item for item in _items]
        super().__init__(sum([t.bits for t in self.items]))
        self.name = f'{tuple_T.__name__}{len(self.items)}_{self.bits}'
        add_define(tuple_T, self.definition[1])

    @property
    def definition(self):
        return [type(self).__name__, [t.definition for t in self.items]] # definition needs the class name to get it from globals and the instance names to get them from the Types

    def _write(self, vals):
        temp = bit()
        for t, v in zip(self.items, vals):
            temp += t.write_bits(v)
        return temp

    def _read(self, bits):
        return [t.read_bits(bits) for t in self.items]


class dict_T(static_encoded_type):
    DEFINED = []
    __name__ = 'dict_T_'
    #def __init__(self, defined):


class list_T(static_encoded_type):
    # note that the _T types should contain n amount of a single static type or a n times repeating set of static types import is they all have to be static so its total length can be known
    DEFINED = []
    __name__ = 'list_T'
    def __init__(self, _items):
        self.items = [item for item in _items]
        super().__init__(sum([t.bits for t in self.items]))
        add_define(list_T, self.definition[1])
        self.name = self.__name__ + f'{len(self.items)}_' if len(self.items) != 1 else '_'
        self.name += str(self.bits)

    @property
    def definition(self):
        return [type(self).__name__, [t.definition for t in self.items]]

    def _write(self, vals:list):
        temp = bit()
        for t, i in zip(self.items, vals):
            temp += t.write_bits(i)
        return temp

    def _read(self, bits):
        return [t.read_bits(bits) for t in self.items]




# Objects/Structs

In [32]:
from random import randint as rng
from datetime import datetime as dt, timedelta as td

class bit_object(object):
    __slots__ = 'name', '_T', '_type', 'bits'
    def __init__(self, _type, _T, bits):
        self.name = _type.__name__
        self._T = _type
        self._type = _T
        self.bits = bits

    def read_bits(self, bits):
        pass

    def write_bits(self, value):
        pass

class RGB(object):
    __name__ = 'RGB'
    __slots__ = 'r', 'g', 'b'
    def __init__(self, rgb=None):
        [self.__setattr__(key, rng(0, 255) if rgb is None else rgb[i]) for i, key in enumerate(self.__slots__)]

    def __repr__(self):
        msg = ' '
        return 'RGB :>|' + '|'.join([str(self.__getattribute__(slot)) for slot in self.__slots__])

class bit_RGB(bit_object):
    def __init__(self):
        super().__init__(RGB, uint(8), 24)

    def read_bits(self, bits):
        return self._T(rgb=[self._type.read_bits(bits) for i in range(3)])

    def write_bits(self, value):
        temp = bit()
        for slot in value.__slots__:
            temp += self._type.write_bits(value.__getattribute__(slot))
        return temp

class bit_Image(bit_object):
    def __init__(self):
class bit_date(bit_object):
    def __init__(self):
        super().__init__(dt, bit_fchar(), 40)

    def read_bits(self, bits):
        temp = ''.join([self._type.read_bits(bits) for i in range(self.bits // self._type.bits)])
        return dt.strptime(temp, '%d.%m.%Y').date()

    def write_bits(self, value):
        temp = bit()
        for char in value.strftime('%d.%m.%Y'):
            temp += self._type.write_bits(char)
        return temp

env = Env('a')
env.declare_object(bit_RGB)
env.declare_object(bit_date)
print(env)
for d in dt.now(), dt.now() - td(days=3):
   env.add_buffer('datetime', d)
bit_data = env.write()
print(bit_data)
print(env.read(bit_data))

Env a:
0. int
1. str
2. bool
3. NoneType
4. float
5. list
6. tuple
7. dict
8. uint_8
9. RGB
10. datetime

bitarray('1010000100011010100100111010000110010001001110100000100010101001001110100001100100010011')
[datetime.date(2024, 4, 22), datetime.date(2024, 4, 19)]


In [ ]:
print([uint(8)] * 3)
t = Env('')
print(t)
print(t.get_type('list_T3_24').write_bits([24, 234, 111]))

In [ ]:
test = sstr(('World', 'abc'))
print(test.write_bits('abc'))
env = Env('test')
env.add_buffer('sstr', 'World')
env.add_buffer('uint_8', 128)
env.add_buffer('int', 12)
env.add_buffer('list_T3_24', [72, 144, 226])
print('--------------------------------------------------------------------------------------------------------------------')
print(env.buffer)
print('-----------------------------------------------------------------------------------------------')
print(env)
#print(env.header)
print(env.write(), len(env.write()))
test = env.write()
print(env.read(test))

#print(env.get_type('list_T3_uint_8').write_bits([11, 123, 222]))

In [ ]:
env = Env('test')
print(env)
print(env.header)

In [ ]:
class static_bit_digit(static_encoded_type):
    __name__ = 'ufloat_'
    DEFINED = []
    def_type = bit_int
    def __init__(self, bits):
        if isinstance(bits, bit):
            bits = static_bit_digit.def_type().read_bits(bits)
        elif type(bits) == int:
            bits *= 4
        super().__init__(bits)
        add_define(static_bit_digit, self.bits)
        self.name = self.__name__ + str(self.bits)

    @property
    def definition(self):
        return static_bit_digit.def_type().write_bits(self.bits)

    def _read(self, bits):
        string = ''
        for char in [str(b2i(bits[i*self.bits:(i+1)*self.bits])) for i in range(len(bits)//self.bits)]:
            string += '.' if char == '10' else '' if char == '15' else char
        return float(string)

    def _write(self, val):
        temp = bit()
        val = str(val).lstrip('-0')
        for char in str(val):
            temp += i2b(10, 4) if char == '.' else i2b(int(char), 4)
        while len(temp) < self.bits:
            temp += i2b(15, 4)
        return temp

TYPE_REF.append(static_bit_digit(8))

class static_bit_sdigit(static_encoded_type):
    __name__ = 'sfloat_'
    DEFINED = []
    def_type = bit_int
    def __init__(self, bits):
        if isinstance(bits, bit):
            bits = static_bit_sdigit.def_type().read_bits(bits)
        elif type(bits) == int:
            bits *= 4
        super().__init__(bits+1)
        add_define(static_bit_sdigit, self.bits)
        self.name = self.__name__ + str(self.bits)

    @property
    def definition(self):
        return static_bit_sdigit.def_type().write_bits(self.bits)

    def _read(self, bits):
        string = '' if b2i(bits.cut(1)) == 0 else '-'
        for char in [str(b2i(bits[i*self.bits:(i+1)*self.bits])) for i in range(len(bits)//self.bits)]:
            string += '.' if char == '10' else '' if char == '15' else char
        return float(string)

    def _write(self, val):
        temp = bit([0]) if val >= 0 else bit([1])
        val = str(val).lstrip('-0')
        for char in str(val):
            temp += i2b(10, 4) if char == '.' else i2b(int(char), 4)
        while len(temp) < self.bits:
            temp += i2b(15, 4)
        return temp

TYPE_REF.append(static_bit_sdigit)

'''
class bit_int(length_encoded_type):
    __name__ = 'int'
    def __init__(self, *args):
        super().__init__(4)

    def _encoding(self, bits:bit):
        return 2 ** b2i(bits) + 1 # returns the length in bits of the integer

    def _read(self, bits:bit):
        sign = bits.cut(1)
        return b2i(bits) if b2i(sign) == 0 else -b2i(bits)

    def _decoding(self, val:int):
        for i, v in enumerate([1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768])
            #for i in range(2**self.bits):
            if 2 ** v >= abs(val):
                return i
        print(f'ERROR {val} is to large')

    def _write(self, value:int, length:int):
        temp = bit([0]) if value >= 0 else bit([1])
        return i2b(length, self.bits) + temp + i2b(abs(value), 2**length)

TYPE_REF.append(bit_int)

## bit_digit is decoding numbers(int/floats) as string with a charset 0-9 and . its the default float for the type system
class bit_digit(endbit_encoded_type):
    __name__ = 'float'
    def __init__(self, *args):
        super().__init__(4, i2b(15, 4))

    def read_bits(self, bits:bit, _return=float):
        bits = super().read_bits(bits)
        string = ''
        for char in [str(b2i(bits[i*self.bits:(i+1)*self.bits])) for i in range(len(bits)//self.bits)]:
            string += char if len(char) == 1 else {'10': '.', '11': '-'}[char]
        return _return(string)

    def write_bits(self, numbers):
        temp = bit()
        for digit in str(numbers):
            if digit in '.-':
                temp += {'.': i2b(10, 4), '-': i2b(11, 4)}[digit]
            else:
                temp += i2b(int(digit), 4)
        return super().write_bits(temp)

TYPE_REF.append(bit_digit)

class bit_literal(static_encoded_type):
    __name__ = 'bool'
    def __init__(self, *args):
        super().__init__(1)

    def _read(self, bits:bit):
        return (False, True)[b2i(bits)]

    def write_bits(self, val):
        return i2b(int(val), self.bits)

TYPE_REF.append(bit_literal)

class bit_none(static_encoded_type):
    __name__ = 'NoneType'
    def __init__(self, *args):
        super().__init__(0)

    def read_bits(self, bits:bit):
        return None

    def write_bits(self, val):
        return bit()

TYPE_REF.append(bit_none)
'''
#TEST(bit_float, (1.2345, .123, -1.23))
#TEST(bit_literal, (True, False))
#TEST(bit_int, (52, 12356, 10**4299, -52, -10**4299))
#TEST(static_bit_int, (123, 255))    # Note that the TEST function has a default definition value of 8 passed

#print(TYPE_REF.get_type_by_key('static_bit_sint'))
#print(TYPE_REF.to_header)
t = sint(8)
print(t.name, t.__name__, type(t).__name__)

In [ ]:
class dummy_pixel(object):
    __slots__ = 'r', 'g', 'b'

    def __init__(self, *args):
        [self.__setattr__(slot, arg) for slot, arg in zip(self.__slots__, args)]

    @staticmethod
    def definition(cls, **kwargs):
        name = type(cls).__name__
        return name, [kwargs.get(slot) for slot in cls.__slots__]



In [ ]:
test = HEAD.write()
print(test)

In [ ]:


class subset_bit_char(static_encoded_type):
    DEFINED = []
    def_type = bit_str
    __name__ = 'chars_'
    def __init__(self, define):
        if isinstance(define, bit):
            chars = subset_bit_char.def_type().read_bits(define)
            key = subset_bit_char.def_type().read_bits(define)
        else:
            chars, key = define
        self.chars = [char for char in chars]
        super().__init__(self._bits)
        self.name = key
        if chr(0) not in self.chars:
            self.chars.insert(0, chr(0))
        add_define(subset_bit_char, (self.chars, self.name))

    @classmethod
    def get_type(cls, key):
        for char, name in subset_bit_char.DEFINED:
            if key == name:
                return cls((char, name))

    @property
    def definition(self):
        temp = bit()
        temp += subset_bit_char.def_type().write_bits(''.join(self.chars))
        temp += subset_bit_char.def_type().write_bits(self.name)
        return temp

    @property
    def _bits(self):
        i = 0
        while 2 ** i < len(self.chars):
            i += 1
        return i

    def _read(self, bits:bit):
        return self.chars[b2i(bits.cut(self.bits))]

    def _write(self, val):
        return i2b(self.chars.index(val), self.bits)

TYPE_REF.append(subset_bit_char)

class static_bit_str(static_encoded_type):
    DEFINED = []
    def_type = bit_int, bit_str
    __name__ = 'str_'
    def __init__(self, define):
        if isinstance(define, bit):
            bits, char_set = [t().read_bits(define) for t in static_bit_str.def_type]
        else:
            bits, char_set = define
        if type(char_set) == str:
            self.chars = subset_bit_char.get_type(char_set)
        else:
            self.chars = char_set
        super().__init__(bits * self.chars.bits)
        add_define(static_bit_str, (bits, self.chars.name))
        self.name = self.__name__ + str(bits) + '_' + self.chars.name

    @property
    def definition(self):
        return static_bit_str.__name__, self.chars.definition

    def _write(self, value):
        while len(value) * self.chars.bits < self.bits:
            value += chr(0)
        temp = bit()
        for i, char in enumerate(value):
            print(char)
            temp += self.chars.write_bits(str(value)[i])
            if i * self.chars.bits == self.bits:
                return temp
        return temp

    def _read(self, bits:bit):
        string = ''
        while len(bits) >= self.chars.bits:
            t = self.chars.read_bits(bits)
            if ord(t) == 0:
                return string
            else:
                string += t # self.chars.read_bits(bits)
        return string


#print(TYPE_REF.type_keys)

c = subset_bit_char(('1234567890.: ', 'datetime'))
static_bit_str((10, c.name))

#TYPE_REF.append(static_bit_str)

#TEST(bit_str, ('Hello', '1line\n2line\n3line', 'balablabalablabalba'))


In [ ]:
#print(TYPE_REF)

In [ ]:
class timeline(list):
    def __init__(self, data):
        if isinstance(data, bit):
            data = self.read_bits(data)
        super().__init__(data)
        self.fill()

    @classmethod
    def read_bits(cls, bits):
        bits = bits.cut(bit_int().read_bits(bits))
        temp = None
        while len(bits) != 0:
            if temp is None:
                temp = [bit_date().read_bits(bits)]
            else:
                temp.append(bit_bool().read_bits(bits))
        return cls(temp)

        #return cls([bit_date().read_bits(bits) if i == 0 else bit_bool().read_bits(bits) for i in range(bit_int().read_bits(bits))])

    def write_bits(self):
        temp = bit_date().write_bits(self[0])
        for b in self[1:]:
            temp += bit_bool().write_bits(b)
        return bit_int().write_bits(len(temp)) + temp

    @property
    def start(self):
        return self[0]

    @property
    def definition(self):
        return bit_date(), bit_bool()

    @property
    def current(self):
        self.fill()
        return self.start + delta(days=len(self))

    def fill(self):
        [self.append(0) for day in range((dt.today().date() - self.start).days - len(self))]

    def ref_data_index(self, date):
        n = 0
        for i in range(len(self)):
            if date.date() == (self.start + delta(days=i)).date():
                self.__setitem__(i, 1)
                return (n, self.count(1))
            n += self[i]

def past_date(days):
    return (dt.now() - delta(days=days)).date()

temp = timeline([past_date(20)])
print(temp)
bits = temp.write_bits()
print(bits)
print(timeline.read_bits(bits))


class dataline(list):
    def __init__(self, arr:list):
        super().__init__(arr)

    def __setitem__(self, i, val):
        i, size = i
        if size > len(self):
            temp = self[:i] + [val] + self[i:]
            super().__setitem__(slice(None, None, 1), temp)
        else:
            temp = self[i] + val
            super().__setitem__(i, temp)

class pageline(dataline):
    def __init__(self, arr:list):
        if len(arr) > 0:
            self.current = sum(arr)
        else:
            self.current = 0
        super().__init__(arr)

    def __setitem__(self, i, val):
        self.current += val
        super().__setitem__(i, val)

class cartline(dataline):
    def __init__(self, arr:list, cur=100):
        self.current = cur
        super().__init__(arr)

    def __setitem__(self, i, val):
        temp = val - self.current if val - self.current >= 0 else 0
        self.current = val
        super().__setitem__(i, temp)

class dataset(dict):
    def __init__(self, **kwargs):
        temp = {'Date': timeline(kwargs.get('Date', [dt.today()]))}
        temp['Page'] = pageline(kwargs.get('Page', []))
        temp['Cart'] = cartline(kwargs.get('Cart', []))
        super().__init__(temp)

    def __repr__(self):
        msg = ''
        for key in ('Date', 'Page', 'Cart'):
            t = ' '.join([str(i) for i in self[key]])
            msg += f'{key} => [{self[key].current}] => {t} \n'
        return msg

    def update(self, **kwargs):
        self['Date'].fill()
        i = self['Date'].ref_data_index(kwargs.get('Date', dt.today()))
        if not i is None:
            for key, val in self.items():
                if key != 'Date':
                    val.__setitem__(i, kwargs.get(key, 0))
        else:
            print(i)
        print(self)
        return self